# Toxicology Testing
For this assignment, you will use a chemical dataset to train a neural network to predict human reaction to exposure to certain compounds.

# Import Packages

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import deepchem as dc
from sklearn.metrics import accuracy_score
from tensorflow import keras
from keras import layers
import datetime

2024-10-05 17:16:04.076057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-05 17:16:04.832312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-05 17:16:04.832382: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-05 17:16:04.832387: W 

# Set Environment

In [2]:
np.random.seed(456)
tf.random.set_seed(456)

# Load and Prep Data

In [3]:
_, (train, valid, test), _ = dc.molnet.load_tox21()
train_X, train_y, train_w = train.X, train.y, train.w
valid_X, valid_y, valid_w = valid.X, valid.y, valid.w
test_X, test_y, test_w = test.X, test.y, test.w
train_y = train_y[:, 0]
valid_y = valid_y[:, 0]
test_y = test_y[:, 0]
train_w = train_w[:, 0]
valid_w = valid_w[:, 0]
test_w = test_w[:, 0]

# Significant Modification
Code originally written for Tensorflow v1, which has been deprecated for years. For this assignment, we will use Tensorflow v2. Re-writing the entire architecture, paying attention to the steps in the assignment, rather than the starter code. Will also be using Keras sequential architectures. One hint comes from [here](https://www.geeksforgeeks.org/hidden-layer-perceptron-in-tensorflow/).

# Define Model Placeholders

In [4]:
d = 1024
n_hidden = 50
learning_rate = .001
n_epochs = 10
batch_size = 100

# Create Model Architecture

In [5]:
model = keras.Sequential([ 
	layers.Dense(256, activation = 'relu', input_shape = [d]), 
	layers.BatchNormalization(), 
	layers.Dense(256, activation = 'relu'), 
	layers.Dropout(0.3), 
	layers.BatchNormalization(), 
	layers.Dense(1, activation = 'sigmoid') 
]) 

# model.compile( 
# 	loss = 'categorical_crossentropy', 
# 	optimizer = 'adam', 
# 	metrics = ['AUC'] 
# ) 

model.compile( 
	loss = 'binary_crossentropy', 
	optimizer = 'adam', 
	metrics = ['AUC'] 
) 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               262400    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 1)                 2

# Tensorboard Setup

In [6]:
# https://www.tensorflow.org/tensorboard/get_started
# Only run this line if not yet loaded
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Implement Mini-Batch Training

In [7]:
model.fit(train_X, train_y, 
          epochs = n_epochs, 
          batch_size = batch_size, 
          verbose = 1, 
          validation_data = (valid_X, valid_y)) 

Epoch 1/10
63/63 [==============================] - 2s 16ms/step - loss: 0.5617 - auc: 0.7279 - val_loss: 0.2884 - val_auc: 0.7417
Epoch 2/10
63/63 [==============================] - 1s 12ms/step - loss: 0.2447 - auc: 0.8742 - val_loss: 0.1596 - val_auc: 0.7076
Epoch 3/10
63/63 [==============================] - 1s 12ms/step - loss: 0.1237 - auc: 0.9294 - val_loss: 0.1443 - val_auc: 0.7396
Epoch 4/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0725 - auc: 0.9753 - val_loss: 0.1592 - val_auc: 0.7469
Epoch 5/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0544 - auc: 0.9794 - val_loss: 0.1591 - val_auc: 0.7188
Epoch 6/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0412 - auc: 0.9832 - val_loss: 0.1708 - val_auc: 0.7621
Epoch 7/10
63/63 [==============================] - 1s 11ms/step - loss: 0.0295 - auc: 0.9917 - val_loss: 0.1731 - val_auc: 0.7477
Epoch 8/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0304 - a

# TensorBoard Results

In [8]:
%tensorboard --logdir logs/fit

Launching TensorBoard...

In [9]:
# train_writer = tf.summary.FileWriter('/tmp/fcnet-tox21',
#                                      tf.get_default_graph())
# N = train_X.shape[0]

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     step = 0

# for epoch in range(n_epochs):
#     pos = 0

# while N > pos:

#       batch_X = train_X[pos:pos+batch_size]
#       batch_y = train_y[pos:pos+batch_size]
#       feed_dict = {x: batch_X, y: batch_y}
#       _, summary, loss = sess.run([train_op, merged, l], feed_dict = feed_dict)
#       print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
#       train_writer.add_summary(summary, step)

#       step += 1
#       pos += batch_size

# # Make Predictions
# valid_y_pred = sess.run(y_pred, feed_dict={x: valid_X})

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2c4d82d2-b129-4486-baa4-05c502f179ef' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>